In [2]:
%matplotlib inline
import sys
sys.path.remove('/Library/Python/2.7/site-packages')
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import cv2
import os
import urllib
import datetime
import re
import time
import matplotlib as mpl
colors = ['#0055A7', '#2C3E4F', '#26C5ED', '#00cc66', '#D34100', '#FF9700', '#091D32']
mpl_update = {'font.size':16,'xtick.labelsize':14,'ytick.labelsize':14,'figure.figsize':[12.0,8.0],
              'axes.labelsize':20,'axes.labelcolor':'#677385',
              'axes.titlesize':20,'lines.color':'#0055A7','lines.linewidth':3,'text.color':'#677385'}
mpl.rcParams.update(mpl_update)
from IPython import display

<img src="figures/svds.png" alt="SVDS" width="100" align="right">

# Trainspotting: real-time detection of a train’s passing from video

### PyCon 2016

### Chloe Mawer | @chloemawer | chloe@svds.com

<img src="figures/caltrain_header.jpg" alt="CaltrainHeader" width="960" height="200">
## The Caltrain obsession


## Caltrain facts
* Commuter rail between San Francisco and San Mateo and Santa Clara counties ~30 stations 
* 118 passenger cars
* 60% >=30 years old
* 2014 weekday ridership is 52,019 people
* No reliable real-time status information
    * API outage between April 5th and June 2nd


## SVDS objective
* Create a mobile app that provides useful real-time system status and prediction information to commuters:
* Use publically available signals to build system that contains the state of train locations:
    * Caltrain API
    * Twitter
    * Rider GPS
* Use HQ’s proximity to Caltrain to directly observe system via audio & video


<center><img src="figures/caltrain_tweet.jpg" alt="Tweet" width="425" style="horizontal-align:middle"></center>

<center><img src="figures/appdemo.gif" alt="AppScreenShots" width="350"></center>

<br />
<br />
<img src="figures/caltrain-sign.jpg" alt="Window" align="right" width="400">
<br />

# Did the train really leave?

<br />
<br />
<img src="figures/train_in_window.jpg" alt="Window" align="right" width="400">
<br />
<br />
<br />
# Did the train really leave?
<br />

# Agenda

<img src="figures/talk-roadmap.png" alt="Window" align="left" width="700">

<img src="figures/opencv.png" alt="OpenCV" width="50" align='right'>
# Motion detection in Python: [OpenCV](http://www.opencv.org)
* Written in optimized C/C++.
* C++, C, Python and Java interfaces.
* Windows, Linux, Mac OS, iOS and Android.
* Free for academic and commercial use.
* Can use multi-core processing.
* Designed for computational efficiency with a strong focus on real-time applications.

* To install, use Anaconda!
    * `conda install -c https://conda.binstar.org/menpo opencv3`

# If you are not obsessed with trains... 
* Many other applications

* Following along

# The original video

In [ ]:
video_file = cv2.VideoCapture('video/orig.mp4')

# Read iterates through the frames in the video object and returns:
# 1. Logical - True if a frame has been read
# 2. Image - an array with the current frame
read_file, frame = video_file.read()
original = []
while read_file:
    # Going to grab the frame and create a list for future use 
    original.append(frame)
    
    # Use imshow to play video
    cv2.imshow('Original video',frame)
    
    # Get next frame 
    read_file, frame = video_file.read()
    
    # Pause and allow for "q" button to stop video
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cv2.destroyWindow('Original video')

In [3]:
# In the case, openCV does not want to open your avi file: 
a = []
b = []
original = []
with open('video/test.avi') as f:
    bytes = f.read()
p = re.compile(r'\xff\xd8')
p2 = re.compile(r'\xff\xd9')
for m in p.finditer(bytes):
    a += [m.start()]
for m in p2.finditer(bytes):
    b += [m.start()]
for c, d in zip(a, b):
    jpg = bytes[c:d+2]
    original.append(cv2.imdecode(np.fromstring(jpg, dtype=np.uint8),1))

In [ ]:
for j, frame in enumerate(original):
    
    cv2.imshow('Original video', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cv2.destroyWindow('Original video')

### Key
* **`frame`**: An image represented by a *M* x *N* x 3 numpy ndarray.
* **`original`**: A list containing a series of frames that make up the video. 
* **`cv2.imshow()`**: Displays an input frame. 
* **`cv2.waitKey()`**: Used to introduce delay between frames.
* **`cv2.destroyWindow()`**: Will close a given window.

<center><h1>The original video</h1></center>
<center><video width="640" height="480" controls>
  <source src="video/orig.mp4" type="video/mp4">
</video></center>

# If you are not obsessed with trains...
Get immediate feedback from your computer's camera! 

In [ ]:
feed = cv2.VideoCapture(0) 

The following will read and display the feed in a window called 'Me' until you press `q` to quit:

In [ ]:
#feed.read()[0] is True unless there is no frame to grab (e.g. camera not working)
while feed.read()[0]:
    # Grab the current frame
    current_frame = feed.read()[1]
    
    # Show the current frame in a window called "Me"
    cv2.imshow('Me', current_frame) 
    
    # Pauses to make computer time = real time
    # and allows pressing "q" on the keyboard
    # to break the loop
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

You must close the window with the following command when done: 

In [ ]:
cv2.destroyWindow('Me')

To discontinue the feed capturing video, you must release it: 

In [ ]:
feed.release()

# Steps for train detection

<ol type="a">
  <li>Is something moving? </li>
  <li>Is that moving thing a train? </li>
  <li>In what direction is it moving? </li>
</ol>

# 1. Is something moving? 

A. Develop a model of the background. <br>
B. Identify pixels in the current frame that do not match the background.

# A. Develop a model of the background

# Convert to one channel

In [ ]:
for j,frame in enumerate(original):
    
    # Convert the frame to one channel (e.g. gray scale)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    cv2.imshow('Gray scale',gray_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cv2.destroyWindow('Gray scale')

<center><h1>Convert to one channel</h1></center>
<center><video width="640" height="480" controls>
  <source src="video/gray.mp4" type="video/mp4">
</video></center>

# Computer Camera

In [ ]:
while True:
    current_frame = feed.read()[1]
    gray_frame = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('Gray scale',gray_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cv2.destroyWindow('Gray scale')

# Smooth the image

Gaussian Blur
<br />
<img src="figures/2dgaussian.png" alt="2D Gaussian" width="250" align="left">
<img src="figures/gaussian-kernel.png" alt="Gaussian kernel" width="250" align="right">
<br />
<br />

\begin{equation}
G(x,y) = Ae^{\frac{-(x-\mu_x)^2}{2\sigma_x^{2}}+\frac{-(y-\mu_y)^2}{2\sigma_y^{2}}}
\end{equation}

Implement in Python: 

`smooth_frame = cv2.GaussianBlur(gray_frame, (kx, ky), 0)`

In [7]:
fig, ax = plt.subplots(2,3,figsize=(16,8))
frame = original[20]
gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
plt.subplot(2,3,1);plt.imshow(gray_frame,cmap='Greys_r'); plt.axis('off'); plt.title('No smoothing');

smooth9 = cv2.GaussianBlur(gray_frame, (9,9), 0)
plt.subplot(2,3,2);plt.imshow(smooth9,cmap='Greys_r'); plt.axis('off'); plt.title('$k_x = k_y = 9$');

smooth15 = cv2.GaussianBlur(gray_frame, (15, 15), 0)
plt.subplot(2,3,3);plt.imshow(smooth15,cmap='Greys_r'); plt.axis('off'); plt.title('$k_x = k_y = 15$');

smooth31 = cv2.GaussianBlur(gray_frame, (31, 31), 0)
plt.subplot(2,3,4);plt.imshow(smooth31,cmap='Greys_r'); plt.axis('off'); plt.title('$k_x = k_y = 31$');

smooth45 = cv2.GaussianBlur(gray_frame, (45,45), 0)
plt.subplot(2,3,5);plt.imshow(smooth45,cmap='Greys_r'); plt.axis('off'); plt.title('$k_x = k_y = 45$');

smooth61 = cv2.GaussianBlur(gray_frame, (61, 61), 0)
plt.subplot(2,3,6);plt.imshow(smooth61,cmap='Greys_r'); plt.axis('off'); plt.title('$k_x = k_y = 61$');
plt.savefig('figures/smooth_images_k.jpg')
plt.close();

<center><h1>Smooth the image</h1></center>

<img src="figures/smooth_images_k.jpg" alt="Smooth images" width="960">

# Smooth the image


In [ ]:
k = 31 # Define Gaussian kernel size

for j, frame in enumerate(original):
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply a Gaussian blur to the gray scale frame 
    smooth_frame = cv2.GaussianBlur(gray_frame, (k, k), 0)
    
    cv2.imshow('Smooth', smooth_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
        
cv2.destroyWindow('Smooth')

# Computer camera

In [ ]:
k = 31
while True:
    current_frame = feed.read()[1]
    gray_frame = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (k, k), 0)
    cv2.imshow('Smooth',smooth_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

In [ ]:
cv2.destroyWindow('Smooth')

<center><h1>Smooth the image</h1></center>

<center><video width="640" height="480" align="center" controls>
  <source src="video/smooth.mp4" type="video/mp4">
</video></center>

# Calculate the running average

At each time step, *t*, calculate at each pixel (x,y): 
\begin{equation}
\mathbf{R}(x,y,t) = (1-\alpha)\mathbf{R}(x,y,t-1) + \alpha\mathbf{F}(x,y,t)
\end{equation}

* $\mathbf{R}:$  Running average 
* $\mathbf{F}:$ Frame being added to the running average
* $\alpha:$ How heavily to weight the new frame

Implement in Python: 

```cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)```

* Running average, *R* at *t* is the weighted sum of the running average at *t-1* and the current frame, *F*. 
* Smoothing in time


# Calculate the running average

In [ ]:
alpha = 0.2 # Define weighting coefficient
running_avg = None # Define variable for running average

k = 31
for j, frame in enumerate(original):
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (k, k), 0)
    
    # If this is the first frame, making running avg current frame
    # Otherwise, update running average with new smooth frame
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
    else:
        cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
    
    cv2.imshow('Running average', cv2.convertScaleAbs(running_avg))

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cv2.destroyWindow('Running average')

# Computer camera

In [ ]:
alpha = 0.02 # Define weighting coefficient
running_avg = None # Define variable for running average

k = 31
while True:
    current_frame = feed.read()[1]
    gray_frame = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (k, k), 0)
    
    # If this is the first frame, making running avg current frame
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
    else:
        cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
        
    cv2.imshow('Running Average', cv2.convertScaleAbs(running_avg))
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

In [ ]:
cv2.destroyWindow('Running Average')

<center><h1>Calculate the running average</h1></center>
<center><video width="640" height="480" align="center" controls>
  <source src="video/running.mp4" type="video/mp4">
</video></center>

# B. Identify pixels in the current frame that do not match the background.

# Calculate the difference 
<img src="figures/bg-fg.jpg" alt="Background and foreground" width="860">
Implement in Python: 

`diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))`

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(16,6))
plt.subplot(1,2,1);
plt.imshow(original[70],cmap='Greys_r'); plt.axis('off'); 
plt.subplot(1,2,2);
plt.imshow(original[73],cmap='Greys_r'); plt.axis('off');  
plt.savefig('bg-fg.jpg'); plt.close()

# Calculate the difference
<img src="figures/diffcolor73.png" alt="Difference frame" width="760">

# Calculate the difference

In [ ]:
alpha = 0.2 
running_avg = None 
k = 31
for j, frame in enumerate(original):

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (k, k), 0)
    
    # If this is the first frame, making running avg current frame
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
        
    # Find |difference| between current smoothed frame and running average
    diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))
    
    # Then add current frame to running average after
    cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)

    cv2.imshow('Difference', diff)

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cv2.destroyWindow('Difference')

# Computer camera

In [ ]:
alpha = 0.02 # Define weighting coefficient
running_avg = None # Define variable for running average

k = 31
while True:
    current_frame = feed.read()[1]
    gray_frame = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (k, k), 0)
    
    # If this is the first frame, making running avg current frame
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
    
    # Find absolute difference between the current smoothed frame and the running average
    diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))
    
    # Then add current frame to running average after
    cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
        
    cv2.imshow('Difference', diff)
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

<center><h1>Calculate the difference</h1><center>
<center><video width="640" height="480" controls>
  <source src="video/colordiff.mp4" type="video/mp4">
</video></center>

# Threshold the difference

Deciding what constitutes motion and what is just noise.
<img src="figures/diffcolor73.png" alt="Difference frame" width="960">

# Threshold the difference


In [26]:
thresh = 35 # All pixels with differences above this value will be set to 1

alpha = 0.2
running_avg = None 
k = 31
for j, frame in enumerate(original):
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (k, k), 0)
    
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
    
    diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))
    cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
    
    # For all pixels with a difference > thresh, turn pixel to 1, otherwise 0
    _, subtracted = cv2.threshold(diff, thresh, 1, cv2.THRESH_BINARY)
    
    cv2.imshow('Thresholded difference', subtracted)
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cv2.destroyWindow('Thresholded difference')

# Computer camera

In [ ]:
alpha = 0.02 # Define weighting coefficient
running_avg = None # Define variable for running average

k = 31
while True:
    current_frame = feed.read()[1]
    gray_frame = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (k, k), 0)
    
    # If this is the first frame, making running avg current frame
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
    
    # Find absolute difference between the current smoothed frame and the running average
    diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))
    
    # Then add current frame to running average after
    cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
    
    # For all pixels with a difference > thresh, turn pixel to 255, otherwise 0
    _, subtracted = cv2.threshold(diff, thresh, 1, cv2.THRESH_BINARY)
    
    cv2.imshow('Difference', subtracted)
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

<center><h1>Threshold the difference</h1></center>
<center><video width="640" height="480" align="center" controls>
  <source src="video/thresh.mp4" type="video/mp4">
</video></center>

# 2. Is that moving thing a train? 

In [12]:
motion_fractions = []

thresh = 35 
alpha = 0.2
running_avg = None 
k = 31
for j, frame in enumerate(original):
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (k,k), 0)
    
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
    
    diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))
    cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
    _, subtracted = cv2.threshold(diff, thresh, 1, cv2.THRESH_BINARY)
    
    # Calculate the fraction of pixels where motion is detected
    # i.e. where subtracted==1
    motion_fraction = (sum(sum(subtracted))/
                       (subtracted.shape[0]*subtracted.shape[1]))

    motion_fractions.append(motion_fraction)
    


# 2. Is that moving thing a train? 

In [15]:
plt.plot(motion_fractions);
plt.xlabel('Frame number');
plt.ylabel('Fraction of frame in motion');
plt.title('Fraction of frame in motion over time');
plt.savefig('figures/red-frame-fraction.jpg');
plt.close()

<img src="figures/frame-fraction.jpg" alt="Fraction in motion" width="860">

# 2. Is that moving thing a train? 

Is enough of the frame in motion? Does this motion last as long as a train? 
* Area threshold
* Duration

In [21]:
plt.plot(motion_fractions);
plt.axhline(y=0.025,color=colors[3]);
plt.xlabel('Frame number');
plt.ylabel('Fraction of frame in motion');
plt.title('Fraction of frame in motion over time');
plt.savefig('figures/green-frame-fraction-with-horizontal.jpg');
plt.close();


<img src="figures/green-frame-fraction-with-horizontal.jpg" alt="Area threshold" width="860">

# 2. Is that moving thing a train? 

In [20]:
plt.plot(motion_fractions);
plt.axhline(y=0.025,color=colors[3]);
plt.axvline(x=71,color=colors[3], linestyle='--');
plt.axvline(x=136,color=colors[3], linestyle='--');
plt.xlabel('Frame number');
plt.ylabel('Fraction of frame in motion');
plt.title('Fraction of frame in motion over time');
plt.savefig('figures/green-frame-fraction-with-horizontal-and-vertical.jpg');
plt.close();

<img src="figures/green-frame-fraction-with-horizontal-and-vertical.jpg" alt="Area threshold" width="860">

# 2. Is that moving thing a train?

In [ ]:
history_length = 50
thresh = 35 
alpha = 0.2
k = 31

frame_thresh = 0.025
running_avg = None 
history = pd.DataFrame(data=[[0,0]],columns=['Fraction','In_motion'])
for j, frame in enumerate(original):
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (g,g), 0)
    
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
    
    diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))
    cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
    
    _, subtracted = cv2.threshold(diff, thresh, 1, cv2.THRESH_BINARY)
    
    motion_fraction = (sum(sum(subtracted))/
                       (subtracted.shape[0]*subtracted.shape[1]))
    
    history.loc[len(history)+1] = [motion_fraction, motion_fraction>frame_thresh]
    
    detect = (history.tail(history_length).In_motion.sum()==history_length)
    
    if detect: 
        print 'Train detected beginning at frame', j-history_length
        # Reset history
        history = pd.DataFrame(data=[[0,0]],columns=['Fraction','In_motion']) 

In [30]:
ax, fig = plt.subplots(figsize=(12,9))
plt.imshow(original[73]); plt.axis('off');
plt.savefig('figures/original73.jpg',dpi=640.0/12);
plt.close()

# 3. In what direction is it moving? 

<img src="figures/original73.jpg" alt="Area threshold" width="860">

# Define regions of interest (ROI)
<img src="figures/roiframe073.jpg" alt="Area threshold" width="860">

# Track activity in ROIs separately 

### Make frames for video of fractions of left and right ROIs in motion 

In [ ]:
from IPython import display
fig, ax = plt.subplots(figsize=(12,9))
plt.xlim([0, 160]);
plt.ylim([0, 1]);
plt.xlabel('Frame number'); 
plt.ylabel('Fraction of ROI in motion');
plt.title('Fraction of ROIs in motion over time');

left_fractions = []
right_fractions = []

thresh = 35 
alpha = 0.2
k = 31

running_avg = None 
for j, frame in enumerate(original):

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (k, k), 0)
    
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
    
    diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))
    cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
    
    # For all pixels with a difference > thresh, turn pixel to 1, otherwise 0
    _, subtracted = cv2.threshold(diff, thresh, 1, cv2.THRESH_BINARY)

    left_area = subtracted[270:400,240:340] 
    right_area = subtracted[270:400,540:640] 
    
    left_fraction = sum(sum(left_area))*1.0/(left_area.shape[0]*left_area.shape[1])
    right_fraction = sum(sum(right_area))*1.0/(right_area.shape[0]*right_area.shape[1])
    
    left_fractions.append(left_fraction)
    right_fractions.append(right_fraction)
    
    plt.plot(range(len(left_fractions)), left_fractions, color=colors[3], label='Left ROI')
    plt.plot(range(len(right_fractions)), right_fractions, color=colors[0], label='Right ROI')
    if j==0:
        plt.legend();
    
    display.clear_output(wait=True)
    display.display(plt.gcf())
    plt.savefig('figures/gree-left-and-right'+str(j).zfill(3)+'.jpg',dpi=640.0/12)
    if cv2.waitKey(1) & 0xFF == ord('q'): 
            break
plt.close();

### Make frames for video of train with ROIs outlined

In [20]:
left_fractions = []
right_fractions = []

thresh = 35 
alpha = 0.2
k = 31

running_avg = None 
for j, frame in enumerate(original):

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (k, k), 0)
    
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
    
    diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))
    cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
    
    # For all pixels with a difference > thresh, turn pixel to 1, otherwise 0
    _, subtracted = cv2.threshold(diff, thresh, 1, cv2.THRESH_BINARY)

    left_area = subtracted[270:400,240:340] 
    right_area = subtracted[270:400,540:640] 
    
    left_fraction = sum(sum(left_area))*1.0/(left_area.shape[0]*left_area.shape[1])
    right_fraction = sum(sum(right_area))*1.0/(right_area.shape[0]*right_area.shape[1])
    
    left_fractions.append(left_fraction)
    right_fractions.append(right_fraction)
    
    cv2.rectangle(frame, (240,270), (340,400), (0, 204,102), thickness=2)
    cv2.rectangle(frame, (540,270), (640,400), (0, 85,167), thickness=2)
    
    ax, fig = plt.subplots(figsize=(12,9))
    plt.imshow(frame); plt.axis('off');
    plt.savefig('figures/roiframe'+str(j).zfill(3)+'.jpg',dpi=640.0/12)
    plt.close()

    if cv2.waitKey(1) & 0xFF == ord('q'): 
            break
plt.close();

### Make final figure for fraction of ROIs in motion

In [ ]:
fig, ax = plt.subplots(figsize=(12,9))
plt.xlim([0, 160]);
plt.ylim([0, 1]);
plt.xlabel('Frame number'); 
plt.ylabel('Fraction of ROI in motion');
plt.title('Fraction of ROIs in motion over time');
plt.plot(left_fractions, color=colors[3], label='Left ROI');
plt.plot(right_fractions, color=colors[0], label='Right ROI');
plt.savefig('figures/left-and-right.jpg')
plt.legend();
plt.close();

# Track activity in ROIs separately (video)

<center><h1>Track activity in ROIs separately</h1></center>
<center><video width="1260" height="460" align="center" controls>
  <source src="video/green-rois-plus-plot.mp4" type="video/mp4">
</video></center>

<center><h1>Real-time direction detection</h1></center>
<img src="figures/left-and-right.jpg" alt="Fraction in motion" width="860">

# Real-time direction detection

1. Initialize a dataframe, `history`.
2. At each frame, *t*, add row to dataframe indicating if left and/or right meets motion threshold 
3. Repeat #2 until motion threshold met on either side for last *T* frames.
3. The side with the detection for all *T* indicates the train's direction.
4. Reinitialize the `history` dataframe to detect future trains.
5. Return to #2     
<img src="figures/left-and-right.jpg" alt="Area threshold" width="520" align='right'>
<img src="figures/train-history.png" alt="Area threshold" width="200" align='left'>


### Final algorithm 

In [ ]:
history_length = 40
area_thresh = 0.05
history = pd.DataFrame(columns=['Detected left','Detected right'])
left_fractions = []
right_fractions = []
running_avg = None 
thresh = 35 
alpha = 0.2
k = 31
for j, frame in enumerate(original):

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (k, k), 0)
    
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
    
    diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))
    cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
    _, subtracted = cv2.threshold(diff, thresh, 1, cv2.THRESH_BINARY)

    left_area = subtracted[270:400,240:340] 
    right_area = subtracted[270:400,540:640] 
    
    left_fraction = sum(sum(left_area))*1.0/(left_area.shape[0]*left_area.shape[1])
    right_fraction = sum(sum(right_area))*1.0/(right_area.shape[0]*right_area.shape[1])
    
    left_fractions.append(left_fraction)
    right_fractions.append(right_fraction)
    
    # Update the history with whether the train detection criteria was met on either side
    history.loc[len(history)+1] = [left_fraction>area_thresh, right_fraction>area_thresh]
    
    # Get how many of last n frames had a train detected for left and right ROIs
    left_cum, right_cum = history.tail(history_length).sum()
    
    # If all of last n frames had train detected on at least one side
    if left_cum >= history_length or right_cum >= history_length: 
        
        # If a train was detected longer on the left, then it is south bound
        if left_cum > right_cum:
            print 'South bound train beginning at frame', j-history_length
        else: 
            print 'North bound train beginning at frame', j-history_length
        
        train = history.tail(history_length).head(10)
        
        # Reset the history to be able to detect a new train
        history = pd.DataFrame(columns=['Detected left','Detected right'])


# Real-time deployment

# Raspberry pi setup
Cost ~ $130
<center><img src="figures/pi-hardware.png" alt="Tweent" width="900" style="horizontal-align:middle"></center>

<center><h1>Detecting motion at night </h1></center>
<center><img src="figures/infra.jpg" alt="Infrared photography" width="450" style="horizontal-align:middle"></center>


<h1> PiCamera python package </h1>

<p>
```python

import time
import picamera
import picamera.array
import cv2


camera = picamera.PiCamera()

# Adjust resolution of the camera
camera.resolution = (1024, 768)

# Adjust framerate of the camera 
camera.framerate = 30 # frames per second

# Save current frame to jpg
camera.capture('image-file.jpg')

# Start live streaming of video
camera.start_preview()

# Flip the camera
camera.vflip = True
camera.hflip = True

# Set the brightness and contrast 
camera.brightness = 60
camera.contrast = 30


```




# Implementing the algorithm

<p>
```python
with picamera.PiCamera() as camera:
    
    camera.start_preview()
    time.sleep(2)
    
    with picamera.array.PiRGBArray(camera) as stream:
        
        camera.capture(stream, format='bgr')
        
        frame = stream.array
        
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        ...
```

* Processing done on the pi to reduce storage needs
* Processed signals sent back to Grafana
* Second signal of microphone for redundancy

# Video quality considerations

May want to reduce frame rate and resolution to be able to run on Raspberry pi --> will need to change parameters.

# Recap of parameters
* `k` : Size of the kernel used for Gaussian blur.

* `area_thresh` : Minimum fraction of ROI that must be "in motion" to be considered train-like.

* `alpha` : How heavily to weigh each new frame in running average.

* `history_length` : Number of frames in motion required to be considered a train.

In [12]:
from matplotlib.mlab import bivariate_normal

X, Y = np.mgrid[-31:31, -31:31]

Z1 = bivariate_normal(X, Y, 5, 5) 

fig, ax = plt.subplots(figsize=(9.5,8))
plt.pcolor(X, Y, Z1);
plt.colorbar();

plt.xlim([-30, 30]);
plt.ylim([-30, 30]);
plt.title('Gaussian with $k_x = k_y = 31$', color='Black'); 
plt.savefig('figures/gaussian-color-k.jpg')
plt.close()

# Resolution + `k`

Change `k` proportionally with the resolution.

<img src="figures/gaussian-color-k.jpg" alt="Gaussian" width="540" align="center">

* Kernel size determines neighborhood to use in computing each pixel
* Want to scale that neighborhood with the size of the image

### Rescaling the frames to 1/8 resolution and reducing number of frames by half

In [14]:
r = 80.0 / original[0].shape[1]
dim = (80, int(original[0].shape[0] * r))
resized = []
for j, frame in enumerate(original):
    resized_frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    if j%2 == 0:
        resized.append(resized_frame)
        cv2.imshow('Resized', resized_frame)
    
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break
cv2.destroyWindow('Resized')

### Producing example figures of new kernel size for reduced resolution

In [16]:
fig, ax = plt.subplots(2,2,figsize=(12,8))
frame = original[20]
gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
resized_frame = resized[10]
gray_resized = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)

smooth31 = cv2.GaussianBlur(gray_frame, (31, 31), 0)
plt.subplot(2,2,1);
plt.imshow(gray_frame,cmap='Greys_r'); plt.axis('off'); 
plt.title('Original image');

plt.subplot(2,2,2);
plt.imshow(gray_resized, cmap='Greys_r'); plt.axis('off'); 
plt.title('1/8 resolution image');

plt.subplot(2,2,3);
plt.imshow(smooth31,cmap='Greys_r'); plt.axis('off'); 
plt.title('Original image, $k_x = k_y = 31$');
smooth_resized = cv2.GaussianBlur(gray_resized, (3,3), 0)

plt.subplot(2,2,4);
plt.imshow(smooth_resized, cmap='Greys_r'); plt.axis('off'); 
plt.title('1/8 resolution image, $k_x = k_y = 3$');
plt.tight_layout();
plt.savefig('figures/sigmag-resolution-k.jpg')
plt.close();

<center><h1>Resolution and <code>k</code></h1>
</center>

<center><img src='figures/sigmag-resolution-k.jpg' alt="Gaussian" width="860" align="center"></center>



### Proving algorithm on reduced resolution and frame reate and adjusted parameters

In [ ]:
history_length = 20
area_thresh = 0.025
history = pd.DataFrame(columns=['Detected left','Detected right'])

resized_left_fractions = []
resized_right_fractions = []
running_avg = None 
thresh = 35 
alpha = 0.36
k = 3

for j, frame in enumerate(resized):

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (k, k), 0)
    
    if running_avg is None:
        running_avg = np.float32(smooth_frame) 
    
    diff = cv2.absdiff(np.float32(smooth_frame), np.float32(running_avg))
    cv2.accumulateWeighted(np.float32(smooth_frame), running_avg, alpha)
    _, subtracted = cv2.threshold(diff, thresh, 1, cv2.THRESH_BINARY)

    left_area = subtracted[34:50,30:42] 
    right_area = subtracted[34:50,68:80] 
        
    left_fraction = sum(sum(left_area))*1.0/(left_area.shape[0]*left_area.shape[1])
    right_fraction = sum(sum(right_area))*1.0/(right_area.shape[0]*right_area.shape[1])
    
    resized_left_fractions.append(left_fraction)
    resized_right_fractions.append(right_fraction)
    # Update the history with whether the train detection criteria was met on either side
    history.loc[len(history)+1] = [left_fraction>area_thresh, right_fraction>area_thresh]
    
    # Get how many of last n frames had a train detected for left and right ROIs
    left_cum, right_cum = history.tail(history_length).sum()
    
    # If all of last n frames had train detected on at least one side
    if left_cum >= history_length or right_cum >= history_length: 
        
        # If a train was detected longer on the left, then it is south bound
        if left_cum > right_cum:
            print 'South bound train beginning at frame', j-history_length
        else: 
            print 'North bound train beginning at frame', j-history_length
        
        train = history.tail(history_length).head(10)
        
        # Reset the history to be able to detect a new train
        history = pd.DataFrame(columns=['Detected left','Detected right'])


### Creating image for ROIs on reduced resolution image

In [159]:
fig, ax = plt.subplots(figsize=(12,10))
resized72 = cv2.resize(original[72], dim, interpolation = cv2.INTER_AREA)
cv2.rectangle(resized72, (30,34), (42,50), (0, 204, 102), thickness=1)
cv2.rectangle(resized72, (68,34), (80,50), (0, 85, 167), thickness=1)
plt.subplot(2,1,1)
plt.imshow(resized72); plt.axis('off');
plt.savefig('figures/resized-roi.jpg');
plt.close();

#### Plotting fraction of ROIs in motion for new resolution/frame rate

In [158]:
fig, ax = plt.subplots(1,2,figsize=(18,6))
plt.subplot(1,2,1)
plt.plot(left_fractions, color=colors[3], label='Left ROI');
plt.plot(right_fractions, color=colors[0], label='Right ROI');
plt.axhline(y=0.05, color=colors[1],label='area_thresh');
plt.xlabel('Frame number');
plt.ylabel('Fraction of ROI in motion');
plt.title('Original');
plt.legend(loc=2, fontsize=17);
plt.subplot(1,2,2)
plt.plot(resized_left_fractions, color=colors[3]);
plt.plot(resized_right_fractions, color=colors[0]);
plt.xlabel('Frame number');
plt.title('1/8 resolution + 1/2 frame rate');
plt.axhline(y=0.025, color=colors[1]);
plt.savefig('figures/original-and-reduced-left-right-thresh.jpg')
plt.close();

<center><h1>Resolution + <code>area_thresh</code></h1></center>

<center><img src="figures/resized-resized-roi.jpg" alt="Resized ROIs" width="300" align="center"></center>

<center><img src="figures/original-and-reduced-left-right-thresh.jpg" alt="Fraction in motion resized and original" width="960" align="center"></center>


# Frame rate + `alpha`
* *Reminder*: `alpha` determines how heavily to weigh new frame in running average.

* As frame rate (*fps*) is lowered, `alpha` should increase. 

* For $fps_{new} =\frac{fps_{old}}{2}$, mathematically:

\begin{equation}
\alpha_{new} = \alpha_{old}(2-\alpha_{old})
\end{equation}

### Get figure for slide to show history length change

In [118]:
plt.plot(left_fractions);
plt.axhline(y=.025,color=colors[3]);
plt.axvline(x=35,color=colors[3], linestyle='--');
plt.axvline(x=68,color=colors[3], linestyle='--');
plt.xlim([0,160])
plt.xlabel('Frame number');
plt.ylabel('Fraction of frame in motion');
plt.title('Fraction of frame in motion over time');
plt.savefig('figures/short-frame-fraction-with-horizontal-and-vertical.jpg');
plt.close();

<center><h1>Frames rate + <code>history_length</code></h1></center>
<center><img src="figures/green-frame-fraction-with-horizontal-and-vertical.jpg" alt="Fraction in motion short" width="860"></center>


<center><h1>Frames rate + <code>history_length</code></h1></center>
<center><img src="figures/short-frame-fraction-with-horizontal-and-vertical.jpg" alt="Fraction in motion short" width="860"></center>


# Frame rate limitations
Requires one frame with train present in only one ROI.

<img src="figures/roiframe073.jpg" alt="Fraction in motion" width="760" align="center">


# What if the light changes?!

* Background subtraction 
    * Frame differencing
        * Good for indoors
        * Very fast computationally
        * Could train threshold for different lighting
    * Alternative: *Adaptive background subtraction*

* Camera settings
    * Can adjust brightness and contrast of camera in real-time

# Gaussian Mixture-based Background/Foreground Segmentation Algorithm (MOG, MOG2)

* Each pixel considered separately
* Background model (mixture of *k* Gaussian models) continually updated
* Threshold is based on the pdf
 

Implement in Python: 

`cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16,detectShadows=False)`

* `history` = length of history to consider in describing the background model. 
* `varThreshold` = square of the number of standard deviations the pixel must be from its mean to be considered not background.

* Will work on straight color image but small scale movement will still likely show up (like leaves rustling)
* Can using [morphological opening](https://en.wikipedia.org/wiki/Opening_(morphology) to remove areas too small to be considered motion for the application at hand. 
* After a time, the image will become the background, won't see end of train - need history to be much longer than length of event trying to detect.

# Gaussian Mixture-based Background/Foreground Segmentation Algorithm (MOG, MOG2)

```python
mask = cv2.createBackgroundSubtractorMOG2(history=100, 
                                          varThreshold=25,
                                          detectShadows=False)
subtracted = frame.apply(mask)

```



In [73]:
mask = cv2.createBackgroundSubtractorMOG2(history=100, 
                                          varThreshold=25,
                                          detectShadows=False)

history_length = 60
area_thresh = 0.02
history = pd.DataFrame(columns=['Detected left','Detected right'])
left_fractions = []
right_fractions = []
running_avg = None 
thresh = 35 
alpha = 0.2
k = 31

extended_original = original[0:60]+original[60::-1]+original[0:60]+original[60::-1]+original

for j, frame in enumerate(extended_original):

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    smooth_frame = cv2.GaussianBlur(gray_frame, (k, k), 0)
    
    kernel = np.ones((11,11),np.uint8)
    
    subtracted = mask.apply(smooth_frame)
    opening = cv2.morphologyEx(subtracted, cv2.MORPH_OPEN, kernel)
    
    left_area = opening[270:400,240:340]/255
    right_area = opening[270:400,540:640]/255
    
    left_fraction = sum(sum(left_area))*1.0/(left_area.shape[0]*left_area.shape[1])
    right_fraction = sum(sum(right_area))*1.0/(right_area.shape[0]*right_area.shape[1])
    
    left_fractions.append(left_fraction)
    right_fractions.append(right_fraction)
    
    # Update the history with whether the train detection criteria was met on either side
    history.loc[len(history)+1] = [left_fraction>area_thresh, right_fraction>area_thresh]
    
    # Get how many of last n frames had a train detected for left and right ROIs
    left_cum, right_cum = history.tail(history_length).sum()
    
    # If all of last n frames had train detected on at least one side
    if left_cum >= history_length or right_cum >= history_length: 
        
        # If a train was detected longer on the left, then it is south bound
        if left_cum > right_cum:
            print 'South bound train beginning at frame', j-history_length
        else: 
            print 'North bound train beginning at frame', j-history_length
        
        train = history.tail(history_length).head(10)
        
        # Reset the history to be able to detect a new train
        history = pd.DataFrame(columns=['Detected left','Detected right'])


In [66]:
plt.plot(left_fractions[-len(original):], label='Left ROI'); 
plt.plot(right_fractions[-len(original):], label='Right ROI'); 
plt.xlabel('Frame number'); plt.ylabel('Fraction of ROI in motion');
plt.title('Fraction of ROIs in motion over time');plt.legend();
plt.savefig('figures/left-right-fractions-mog2-extended.jpg');
plt.close()

In [68]:
plt.plot(left_fractions, label='Left ROI'); 
plt.plot(right_fractions, label='Right ROI'); 
plt.xlabel('Frame number'); plt.ylabel('Fraction of ROI in motion');
plt.title('Fraction of ROIs in motion over time');plt.legend();
plt.savefig('figures/left-right-fractions-mog2-original.jpg');
plt.close()

<center><img src="figures/left-right-fractions-mog2-100.jpg" alt="MOG2 left right fractions" width="660" style="horizontal-align:middle"></center>

# Must increase the history length to detect longer last objects

```python
mask = cv2.createBackgroundSubtractorMOG2(history=400, 
                                          varThreshold=25,
                                          detectShadows=False)
subtracted = frame.apply(mask)

```

In [40]:
# This time with a longer history to maintain detection 
# through length of train
mask = cv2.createBackgroundSubtractorMOG2(history=400, 
                                          varThreshold=25,
                                          detectShadows=False)

history_length = 60
area_thresh = 0.02
history = pd.DataFrame(columns=['Detected left','Detected right'])
left_fractions = []
right_fractions = []
running_avg = None 
thresh = 35 
alpha = 0.2
k = 31
for j, frame in enumerate(original):

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     gray_frame = gamma_correction(gray_frame.astype('uint8'),0.75)
    smooth_frame = cv2.GaussianBlur(gray_frame, (k, k), 0)
    
    kernel = np.ones((11,11),np.uint8)
    
    subtracted = mask.apply(smooth_frame)
    opening = cv2.morphologyEx(subtracted, cv2.MORPH_OPEN, kernel)
    cv2.imwrite('mog2'+str(j).zfill(3)+'.jpg',opening)
    left_area = opening[270:400,240:340]/255
    right_area = opening[270:400,540:640]/255
    
    left_fraction = sum(sum(left_area))*1.0/(left_area.shape[0]*left_area.shape[1])
    right_fraction = sum(sum(right_area))*1.0/(right_area.shape[0]*right_area.shape[1])
    
    left_fractions.append(left_fraction)
    right_fractions.append(right_fraction)
    
    # Update the history with whether the train detection criteria was met on either side
    history.loc[len(history)+1] = [left_fraction>area_thresh, right_fraction>area_thresh]
    
    # Get how many of last n frames had a train detected for left and right ROIs
    left_cum, right_cum = history.tail(history_length).sum()
    
    # If all of last n frames had train detected on at least one side
    if left_cum >= history_length or right_cum >= history_length: 
        
        # If a train was detected longer on the left, then it is south bound
        if left_cum > right_cum:
            print 'South bound train beginning at frame', j-history_length
        else: 
            print 'North bound train beginning at frame', j-history_length
        
        train = history.tail(history_length).head(10)
        
        # Reset the history to be able to detect a new train
        history = pd.DataFrame(columns=['Detected left','Detected right'])


<center><img src="figures/left-right-fractions-mog2-extended.jpg" alt="MOG2 left right fractions" width="660" style="horizontal-align:middle"></center>

<center><h3>cmawer.github.io/trainspotting </h3></center>
<center><img src="figures/thankyou.png" alt="End" width="500"></center>
<center><h3>@chloemawer | chloe@svds.com  </h3></center>


# Appendix

# Architecture

<center><img src="figures/pi-arch.png" alt="Tweent" width="900" style="horizontal-align:middle"></center>